# Coder Agent Workspace

Drive the lightweight `CoderAgent` to spin up focused Python projects with your local tool-calling LLM.

## Imports & configuration

Set the endpoint, model, and folders once, then reuse the agent across briefs.

In [7]:
from agent import CoderAgent
from pathlib import Path
import json

BASE_URL = "http://localhost:1234/v1"
MODEL = "qwen/qwen3-coder-30b"
PROJECT_ROOT = "generated_projects"
LOG_DIR = "logs"

print(f"Base URL: {BASE_URL}")
print(f"Model: {MODEL}")
print(f"Project root: {PROJECT_ROOT}")
print(f"Log dir: {LOG_DIR}")

Base URL: http://localhost:1234/v1
Model: qwen/qwen3-coder-30b
Project root: generated_projects
Log dir: logs


## Instantiate the coder agent

This keeps the workspace isolated and exposes filesystem tools to the LLM.

In [8]:
coder = CoderAgent(
    base_url=BASE_URL,
    model=MODEL,
    log_dir=LOG_DIR,
    project_root=PROJECT_ROOT,
    verbose=False,
)

print("Coder agent ready.")

Coder agent ready.


## Set the project brief

Edit the text and rerun the next cells whenever you need a fresh build.

In [9]:
project_brief = """Build a minimal CLI that tracks todos in a JSON file.
Include a README with usage instructions."""
project_name = "todo_cli"  # Set to None to auto-generate from the brief
max_iterations = 6

print("Current project name:", project_name)
print(project_brief)

Current project name: todo_cli
Build a minimal CLI that tracks todos in a JSON file.
Include a README with usage instructions.


## Build the project

The agent loops through chat + tool calls until it finishes or hits the iteration cap.

In [10]:
coder.reset_conversation()
summary = coder.build_project(
    project_description=project_brief,
    project_name=project_name,
    max_iterations=max_iterations,
)
print("Build summary:", summary)

Build summary: I've successfully created a complete CLI todo tracking application with the following structure:

## Files Created:
1. `main.py` - The main application that handles CLI commands and JSON storage
2. `README.md` - Complete usage instructions with examples

## Features Implemented:
- Add new todos with `python main.py add "Todo item"`
- List all todos with completion status using `python main.py list`
- Mark todos as complete using `python main.py complete <id>`
- Delete todos using `python main.py delete <id>`
- Todos are persisted in a JSON file called `todos.json`

## How to Use:
1. Run `python main.py add "Buy groceries"` to add a new todo
2. Run `python main.py list` to see all todos
3. Run `python main.py complete 1` to mark todo #1 as complete
4. Run `python main.py delete 1` to remove todo #1

The application uses only Python standard library modules and stores data in a local JSON file. The README includes comprehensive usage instructions with example sessions.

Th

## Inspect generated files

List everything inside the active workspace.

In [11]:
workspace = getattr(coder, "active_workspace", None)
if not workspace:
    print("No active workspace yet.")
else:
    root = workspace.root
    print(f"Workspace root: {root}")
    for path in sorted(root.rglob("*")):
        rel = path.relative_to(root)
        prefix = "[dir] " if path.is_dir() else " [f] "
        print(f"{prefix}{rel}")

Workspace root: /Users/Khaled.Alabsi/projects/llm-agent/generated_projects/todo-cli
 [f] README.md
 [f] main.py


## Peek at a file

Change `target_file` to inspect other outputs from the build.

In [12]:
target_file = "README.md"
    workspace = getattr(coder, "active_workspace", None)
    if not workspace:
        print("No active workspace yet.")
    else:
        file_path = workspace.root / target_file
        if not file_path.exists():
            print(f"{target_file} does not exist in the workspace.")
        else:
            print(f"Contents of {target_file}:
")
            print(file_path.read_text())

IndentationError: unexpected indent (2343194132.py, line 2)

## Review recent log files

Every request/response pair is captured for debugging.

In [ ]:
log_dir = Path(LOG_DIR)
if not log_dir.exists():
    print("No logs yet.")
else:
    files = sorted(log_dir.glob("*.json"))[-10:]
    if not files:
        print("No logs saved yet.")
    else:
        for path in files:
            print(f"{path.name}	{path.stat().st_size} bytes")

## Helper: run new briefs quickly

Use this function in ad-hoc cells to kick off additional builds without reconfiguring anything.

In [ ]:
def run_brief(description: str, *, name: str | None = None, iterations: int = 6):
    coder.reset_conversation()
    result = coder.build_project(description, project_name=name, max_iterations=iterations)
    print(result)
    return result

# Example usage:
# run_brief("Generate a simple stopwatch CLI with tests", name="stopwatch")